# Mapping Data with Folium

![](images/folium.png)

Below, we will go through a brief introduction to the **Folium** library.  This is a nice way to build interactive visuzlizations.  We will be executing these in the jupyter notebooks, however they are easily output as `.html` files ready to be served.  To begin, let's make sure we have folium installed.  

In [1]:
import folium
import pandas as pd

In [2]:
!pip install folium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


We can make basic maps centered at any geolocation.  For example, below we create a basic map of Portland, Oregon.  

In [3]:
m = folium.Map(location=[40.6782, -73.9442])

In [4]:
m

In [5]:
m.save('index_map.html')

We can add arguments that include changing the style of the map and the initial zoom level.

In [6]:
folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

We can use the `popup` argument to include information to be displayed at specified marker locations.

In [7]:
tooltip = 'Click me!'
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)



folium.Marker([45.3288, -121.6625], popup='<i>Mt. Hood Meadows</i>').add_to(m)
folium.Marker([45.3311, -121.7113], popup='<b>Timberline Lodge</b>').add_to(m)
m

We can even include `markdown` syntax and icons.

In [8]:
m = folium.Map(
    location=[45.372, -121.6972],
    zoom_start=12,
    tiles='Stamen Terrain'
)

folium.Marker(
    location=[45.3288, -121.6625],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[45.3311, -121.7113],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[45.3300, -121.6823],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

In [9]:
m

We can manually control radii for markers of interest.  Below, we plot two circles at specific locations.

In [10]:
m = folium.Map(
    location=[45.5236, -122.6750],
    tiles='Stamen Toner',
    zoom_start=13
)

folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup='The Waterfront',
    color='crimson',
    fill=False,
).add_to(m)

folium.CircleMarker(
    location=[45.5215, -122.6261],
    radius=50,
    popup='Laurelhurst Park',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)


In [11]:
m

## Problem

In [12]:
gardens = pd.read_json('https://data.cityofnewyork.us/resource/yes4-7zbb.json')

In [13]:
gardens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 17 columns):
address             535 non-null object
bbl                 431 non-null object
bin                 431 non-null object
boro                536 non-null object
census_tract        429 non-null float64
community_board     536 non-null object
council_district    495 non-null float64
cross_streets       464 non-null object
garden_name         536 non-null object
jurisdiction        536 non-null object
latitude            429 non-null float64
longitude           429 non-null float64
neighborhoodname    323 non-null object
nta                 431 non-null object
postcode            431 non-null object
propid              536 non-null object
size                536 non-null object
dtypes: float64(4), object(13)
memory usage: 71.3+ KB


In [14]:
garden_map = gardens[(gardens.latitude.isna() == False) & (gardens.longitude.isna() == False)]

In [15]:
garden_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429 entries, 0 to 535
Data columns (total 17 columns):
address             429 non-null object
bbl                 429 non-null object
bin                 429 non-null object
boro                429 non-null object
census_tract        429 non-null float64
community_board     429 non-null object
council_district    403 non-null float64
cross_streets       368 non-null object
garden_name         429 non-null object
jurisdiction        429 non-null object
latitude            429 non-null float64
longitude           429 non-null float64
neighborhoodname    265 non-null object
nta                 429 non-null object
postcode            429 non-null object
propid              429 non-null object
size                429 non-null object
dtypes: float64(4), object(13)
memory usage: 60.3+ KB


In [16]:
m = folium.Map(location = [garden_map.latitude[0], garden_map.longitude[0]], tiles='Stamen Toner',)
m

In [17]:
for index, row in garden_map.iterrows():
    folium.CircleMarker(location=(row["latitude"],
                                  row["longitude"]),
                        radius = 1.0,
                        color='blue',
                        fill=True).add_to(m)

In [18]:
m

- What information should be included about the gardens?
The name of the garden should be included over the points
- Can you alter the loop above to have this information available by scrolling over a point?

In [19]:
#read in data
gardens = pd.read_json('https://data.cityofnewyork.us/resource/yes4-7zbb.json')

#subset to where there are latitude and longitudes
garden_map = gardens[(gardens.latitude.isna() == False) & (gardens.longitude.isna() == False)]

#create lists of locations and names
locationlist = garden_map[["latitude","longitude"]].values.tolist()
labels = garden_map["garden_name"].values.tolist()

#create base map and add one marker at first garden
location = [garden_map['latitude'][0], garden_map.longitude[0]]
m = folium.Map(location=location, zoom_start=14)

#loop through and add markers
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)

m

### Mapping Bike Data

Now, we will use a dataset from NYC's citibike data.  Our goal is to compare incoming and outgoing traffic at given stations depending on the time of day.  

In [20]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map)

In [21]:
folium_map

In [22]:
bikes = pd.read_csv('data/201306-citibike-tripdata.csv')

FileNotFoundError: File b'data/201306-citibike-tripdata.csv' does not exist

In [ ]:
bikes.info()

In [ ]:
bikes['starttime'] = pd.to_datetime(bikes['starttime'])
bikes['stoptime'] = pd.to_datetime(bikes['stoptime'])
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
bikes['ehour'] = bikes['stoptime'].map(lambda x: x.hour)

In [ ]:
bikes.head()

In [ ]:
locations = bikes.groupby('start station id').first()

In [ ]:
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]

In [ ]:
subset = bikes[bikes["hour"]==10]

In [ ]:
dept_counts = subset.groupby("start station id").count()

In [ ]:
dept_counts = dept_counts.iloc[:, [0]]

In [ ]:
dept_counts.columns = ["Departure Counts"]

### Problem

Repeat the above for arrivals, in anticipation of joining the two for our map.

In [ ]:
bikes['hour'] = bikes['starttime'].map(lambda x: x.hour)
locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]
subset = bikes[bikes["hour"]==18]
dept_counts = subset.groupby("start station id").count()
dept_counts = dept_counts.iloc[:, [0]]
dept_counts.columns = ["Departure Counts"]


locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["ehour"]==18]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]

In [ ]:
trip_counts = dept_counts.join(locations).join(arr_counts)

In [ ]:
trip_counts.head()

In [ ]:
for index, row in trip_counts.iterrows():
    
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    
    radius = net_departures/7
    
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(folium_map)

In [ ]:
folium_map

In [ ]:
popup_text = """{}<br>
                total departures: {}<br> 
                total arrivals: {}<br>
                net departures: {}"""


popup_text = popup_text.format(row["start station name"],
                               row["Arrival Counts"],
                               row["Departure Counts"],
                               net_departures)

In [ ]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Counts"]-row["Arrival Counts"])
    radius = net_departures/7
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    folium.CircleMarker(location=(row["start station latitude"],
                                  row["start station longitude"]),
                        radius=radius,
                        color=color,
                        fill=True, popup = popup_text).add_to(folium_map)

In [ ]:
folium_map

### PROBLEM

Compare this image to that of when people are leaving work.  Doe you see what you expect?  What does this tell you about movement in the city?

In [ ]:
locations2 = bikes.groupby('end station id').first()
locations2 = locations2.loc[:, ["end station latitude", "end station longitude", "end station name"]]
subset = bikes[bikes["hour"]==10]
arr_counts = subset.groupby("end station id").count()
arr_counts = arr_counts.iloc[:, [0]]
arr_counts.columns = ["Arrival Counts"]